# Most Popular videogames and its social networks
Find the 50 most popular videogames in IGDB and find the social links of each of the games

## 0.- Imports and Environment

In [59]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
import requests
from datetime import datetime
from bs4 import BeautifulSoup
import re
import time

## 1.- API
### 1.1- TEST API
Find an API and create a conection
https://api-v3.igdb.com/

My API connection

In [60]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0',
    'user-Key': 'bc999c8df04132a584fdc01353c07edf',
    'Accept': 'application/json'
}


Test API with the most popular games stats

In [61]:
#path
path ='https://api-v3.igdb.com/'

In [62]:
#endpoint
endpoint = 'games'

In [63]:
# parameters: fields all 
parameters = '?fields=*'
url = ('%s%s%s' % (path,endpoint,parameters))

In [64]:
response = requests.request('POST', url, headers = headers)
results = response.json()
test = json_normalize(results)

In [65]:
test.head()

,aggregated_rating,aggregated_rating_count,alternative_names,category,collection,cover,created_at,external_games,first_release_date,franchise,...,slug,storyline,summary,tags,themes,total_rating,total_rating_count,updated_at,url,websites
0,NaN,NaN,NaN,0,NaN,NaN,1515715200,"[151316, 1193389]",NaN,NaN,...,stick-fighter-ii,NaN,NaN,NaN,NaN,NaN,NaN,1515715200,https://www.igdb.com/games/stick-fighter-ii,NaN
1,NaN,NaN,NaN,0,NaN,NaN,1520899200,[305676],NaN,NaN,...,asian-riddles-4,NaN,NaN,NaN,NaN,NaN,NaN,1520899200,https://www.igdb.com/games/asian-riddles-4,NaN
2,NaN,NaN,"[9240, 9241]",0,2156.0,28681.0,1501027200,[138210],7.388928e+08,814.0,...,segasonic-the-hedgehog,"Sonic, Ray and Mighty have been kidnapped by t...",SegaSonic the Hedgehog is an isometric platfor...,"[268435464, 268435489, 536871049, 536871208, 5...",NaN,NaN,NaN,1546646400,https://www.igdb.com/games/segasonic-the-hedgehog,"[53034, 53035]"
3,NaN,NaN,[18910],2,768.0,54950.0,1507075200,"[135416, 1193409]",1.196294e+09,1173.0,...,sengoku-basara-2-heroes,NaN,Sengoku BASARA 2 Heroes (aka. Sengoku Basara 2...,NaN,NaN,NaN,NaN,1513468800,https://www.igdb.com/games/sengoku-basara-2-he...,NaN
4,NaN,NaN,NaN,0,NaN,NaN,1521072000,"[128240, 1189096]",NaN,NaN,...,racing-live,NaN,NaN,NaN,NaN,NaN,NaN,1521072000,https://www.igdb.com/games/racing-live,NaN


### 1.2- RETRIEVE INFO FROM THE API

Once I have tested the API I'm going to retreive the info I want.

**INFO: Information about the 50 most popular videogames in the IGDB (igdb.com)**

In [66]:
# api fields https://igdb.github.io/api/endpoints/game/
# parameters: fields all / orderby popularity / retrieve 50 results
parameters = '?fields=*&order=popularity:desc&limit=50'
url = ('%s%s%s' % (path,endpoint,parameters))

In [67]:
response = requests.request('POST', url, headers = headers)
results = response.json()
data = json_normalize(results)

Let's check the test Dataframe to understand better the data

In [68]:
data.head()

,age_ratings,aggregated_rating,aggregated_rating_count,alternative_names,artworks,bundles,category,collection,cover,created_at,...,summary,tags,themes,time_to_beat,total_rating,total_rating_count,updated_at,url,videos,websites
0,NaN,70.500000,14.0,NaN,"[6937, 7120, 7121, 7122, 7123, 7124, 7125, 712...",NaN,0,NaN,74152,1457308800,...,The Sinking City is a game of investigation ge...,"[1, 19, 38, 43, 268435487, 536872032, 53687220...","[1, 19, 38, 43]",NaN,70.119493,24.0,1563926400,https://www.igdb.com/games/the-sinking-city,"[19026, 20710, 21441, 21777, 26914, 27511]","[57637, 77115, 77116, 87826, 87986, 107560, 10..."
1,[24721],93.333333,5.0,"[22447, 22448, 22449]",NaN,NaN,0,303.0,76781,1484784000,...,Fire Emblem: Three Houses is being developed f...,"[17, 39, 268435468, 268435472, 268435480, 2684...","[17, 39]",NaN,93.333333,5.0,1564012800,https://www.igdb.com/games/fire-emblem-three-h...,"[20659, 24912, 28487, 28670]","[81147, 104446, 104447, 104448]"
2,[6032],NaN,NaN,NaN,"[4939, 4940, 4941, 4942, 5034, 5035, 5036, 503...",NaN,0,NaN,75012,1357948800,...,The upcoming RPG from CD Projekt RED based on ...,"[18, 33, 38, 268435468, 536871015, 536871018, ...","[18, 33, 38]",NaN,NaN,NaN,1563926400,https://www.igdb.com/games/cyberpunk-2077,"[2743, 20675, 21892, 27840]","[360, 57810, 57811, 57812, 57813, 108258]"
3,[24567],86.636364,12.0,NaN,NaN,NaN,0,240.0,71673,1550016000,...,Build and play the Super Mario courses of your...,"[1, 268435464]",[1],NaN,88.422354,28.0,1564012800,https://www.igdb.com/games/super-mario-maker-2,"[24906, 27269]","[105880, 107982, 107983]"
4,[24853],NaN,NaN,[24817],NaN,NaN,0,NaN,75103,1485388800,...,Marvel's Avengers is an action-adventure game ...,"[1, 17, 18, 268435461, 268435487, 536870926, 5...","[1, 17, 18]",NaN,NaN,NaN,1563926400,https://www.igdb.com/games/marvels-avengers,"[27999, 28021]","[107829, 108417, 108423, 108424, 108443, 108444]"


In [10]:
data.shape

(50, 47)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 47 columns):
age_ratings                17 non-null object
aggregated_rating          15 non-null float64
aggregated_rating_count    15 non-null float64
alternative_names          12 non-null object
artworks                   17 non-null object
bundles                    2 non-null object
category                   50 non-null int64
collection                 14 non-null float64
cover                      50 non-null int64
created_at                 50 non-null int64
expansions                 3 non-null object
external_games             40 non-null object
first_release_date         47 non-null float64
franchise                  8 non-null float64
franchises                 8 non-null object
game_engines               12 non-null object
game_modes                 49 non-null object
genres                     50 non-null object
hypes                      29 non-null float64
id                      

In [12]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
aggregated_rating,15.0,6.358820e+01,2.884833e+01,0.000000e+00,5.950000e+01,7.050000e+01,8.062500e+01,9.333333e+01
aggregated_rating_count,15.0,6.133333e+00,6.822931e+00,0.000000e+00,2.500000e+00,3.000000e+00,7.500000e+00,2.600000e+01
category,50.0,4.000000e-02,2.828427e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00
collection,14.0,1.749214e+03,1.628546e+03,6.200000e+01,2.530000e+02,1.821000e+03,2.405250e+03,4.740000e+03
cover,50.0,5.884936e+04,2.265019e+04,1.016400e+04,4.590275e+04,7.094250e+04,7.456025e+04,7.678100e+04
created_at,50.0,1.488943e+09,4.450275e+07,1.357949e+09,1.470874e+09,1.493294e+09,1.515240e+09,1.559434e+09
first_release_date,47.0,1.482854e+09,1.406166e+08,1.037318e+09,1.487765e+09,1.539734e+09,1.561637e+09,1.589501e+09
franchise,8.0,3.546250e+02,3.168172e+02,2.400000e+01,5.725000e+01,3.375000e+02,5.312500e+02,8.080000e+02
hypes,29.0,4.375862e+01,1.096067e+02,1.000000e+00,3.000000e+00,9.000000e+00,3.800000e+01,5.760000e+02
id,50.0,4.926768e+04,3.854727e+04,1.877000e+03,2.242300e+04,2.831650e+04,8.047050e+04,1.192070e+05


In [13]:
data.columns

Index(['age_ratings', 'aggregated_rating', 'aggregated_rating_count',
       'alternative_names', 'artworks', 'bundles', 'category', 'collection',
       'cover', 'created_at', 'expansions', 'external_games',
       'first_release_date', 'franchise', 'franchises', 'game_engines',
       'game_modes', 'genres', 'hypes', 'id', 'involved_companies', 'keywords',
       'multiplayer_modes', 'name', 'parent_game', 'platforms',
       'player_perspectives', 'popularity', 'pulse_count', 'rating',
       'rating_count', 'release_dates', 'screenshots', 'similar_games', 'slug',
       'status', 'storyline', 'summary', 'tags', 'themes', 'time_to_beat',
       'total_rating', 'total_rating_count', 'updated_at', 'url', 'videos',
       'websites'],
      dtype='object')

Let's keep only the data we need

### 1.3- DATA MANIPULATION


In [50]:
games = data[['id','name','popularity','total_rating','first_release_date','url']]

In [51]:
games.head()

,id,name,popularity,total_rating,first_release_date,url
0,18225,The Sinking City,1342.489026,70.119493,1.561594e+09,https://www.igdb.com/games/the-sinking-city
1,26845,Fire Emblem: Three Houses,921.876817,93.333333,1.564099e+09,https://www.igdb.com/games/fire-emblem-three-h...
2,1877,Cyberpunk 2077,752.805081,NaN,1.586995e+09,https://www.igdb.com/games/cyberpunk-2077
3,115276,Super Mario Maker 2,661.996011,88.422354,1.561680e+09,https://www.igdb.com/games/super-mario-maker-2
4,26950,Marvel's Avengers,651.855153,NaN,1.589501e+09,https://www.igdb.com/games/marvels-avengers


From the unix time create a time date column

In [52]:
#Function to convert unix time stamp into datetime
def convert_unix(ts):
    if ts != ts:
        return np.nan
    else:
        return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [53]:
converted = games['first_release_date'].apply(convert_unix)
games.loc[:,'date_time']  = converted

In [54]:
games.head()

,id,name,popularity,total_rating,first_release_date,url,date_time
0,18225,The Sinking City,1342.489026,70.119493,1.561594e+09,https://www.igdb.com/games/the-sinking-city,2019-06-27
1,26845,Fire Emblem: Three Houses,921.876817,93.333333,1.564099e+09,https://www.igdb.com/games/fire-emblem-three-h...,2019-07-26
2,1877,Cyberpunk 2077,752.805081,NaN,1.586995e+09,https://www.igdb.com/games/cyberpunk-2077,2020-04-16
3,115276,Super Mario Maker 2,661.996011,88.422354,1.561680e+09,https://www.igdb.com/games/super-mario-maker-2,2019-06-28
4,26950,Marvel's Avengers,651.855153,NaN,1.589501e+09,https://www.igdb.com/games/marvels-avengers,2020-05-15


In [55]:
games = games.drop(columns='first_release_date')

In [56]:
games.columns

Index(['id', 'name', 'popularity', 'total_rating', 'url', 'date_time'], dtype='object')

In [57]:
games = games[['id', 'name', 'popularity', 'total_rating', 'date_time', 'url']]

In [58]:
games.head()

,id,name,popularity,total_rating,date_time,url
0,18225,The Sinking City,1342.489026,70.119493,2019-06-27,https://www.igdb.com/games/the-sinking-city
1,26845,Fire Emblem: Three Houses,921.876817,93.333333,2019-07-26,https://www.igdb.com/games/fire-emblem-three-h...
2,1877,Cyberpunk 2077,752.805081,NaN,2020-04-16,https://www.igdb.com/games/cyberpunk-2077
3,115276,Super Mario Maker 2,661.996011,88.422354,2019-06-28,https://www.igdb.com/games/super-mario-maker-2
4,26950,Marvel's Avengers,651.855153,NaN,2020-05-15,https://www.igdb.com/games/marvels-avengers


## 2.- WEB SCRAPPING
### 2.1- TEST WEB SCRAPPING
Test the URL of the first game and find the social accounts
https://www.igdb.com/games/{game}

In [23]:
url = 'https://www.igdb.com/games/the-sinking-city'
html = requests.get(url).content
soup = BeautifulSoup(html,"lxml")
socials = soup.find_all(['a'],{'class':['gamepage-website-link col-md-4']})
socials

[<a class="gamepage-website-link col-md-4" data-reactid="57" href="https://www.youtube.com/channel/UCciY3FqSxrfKQ2nMDUrlf4w" rel="noopener noreferrer nofollow" target="_blank"><span class="fa fa-youtube mar-md-right" data-reactid="58"></span><span data-reactid="59">YouTube channel</span></a>,
 <a class="gamepage-website-link col-md-4" data-reactid="60" href="https://www.facebook.com/TheSinkingCityGame" rel="noopener noreferrer nofollow" target="_blank"><span class="fa fa-facebook mar-md-right" data-reactid="61"></span><span data-reactid="62">Facebook page</span></a>,
 <a class="gamepage-website-link col-md-4" data-reactid="63" href="https://twitter.com/thesinkingcity" rel="noopener noreferrer nofollow" target="_blank"><span class="fa fa-twitter mar-md-right" data-reactid="64"></span><span data-reactid="65">Twitter profile</span></a>,
 <a class="gamepage-website-link col-md-4" data-reactid="66" href="https://thesinkingcity.com/" rel="noopener noreferrer nofollow" target="_blank"><span c

In [24]:
names = [social.text.strip().split("</a>")[0] for social in socials]
names

['YouTube channel',
 'Facebook page',
 'Twitter profile',
 'Official Website',
 'Wikipedia page',
 'Instagram profile',
 'Subreddit']

In [25]:
links = []
for tag in socials:
    link = tag.get('href')
    if link is not None:
        if link.startswith("http"):
            links.append(link)
links

['https://www.youtube.com/channel/UCciY3FqSxrfKQ2nMDUrlf4w',
 'https://www.facebook.com/TheSinkingCityGame',
 'https://twitter.com/thesinkingcity',
 'https://thesinkingcity.com/',
 'https://wikipedia.org/wiki/The_Sinking_City',
 'https://www.instagram.com/frogwares',
 'https://www.reddit.com/r/TheSinkingCity']

In [26]:
gl_df = pd.DataFrame(links).T

In [27]:
gl_df.columns = names

In [28]:
gl_df

,YouTube channel,Facebook page,Twitter profile,Official Website,Wikipedia page,Instagram profile,Subreddit
0,https://www.youtube.com/channel/UCciY3FqSxrfKQ...,https://www.facebook.com/TheSinkingCityGame,https://twitter.com/thesinkingcity,https://thesinkingcity.com/,https://wikipedia.org/wiki/The_Sinking_City,https://www.instagram.com/frogwares,https://www.reddit.com/r/TheSinkingCity


In [29]:
gl_df['url'] = url

In [30]:
gl_df

,YouTube channel,Facebook page,Twitter profile,Official Website,Wikipedia page,Instagram profile,Subreddit,url
0,https://www.youtube.com/channel/UCciY3FqSxrfKQ...,https://www.facebook.com/TheSinkingCityGame,https://twitter.com/thesinkingcity,https://thesinkingcity.com/,https://wikipedia.org/wiki/The_Sinking_City,https://www.instagram.com/frogwares,https://www.reddit.com/r/TheSinkingCity,https://www.igdb.com/games/the-sinking-city


### 2.2- WEB SCRAPING
Scrap all the urls of the popular games and find the social accounts
https://www.igdb.com/games/{game}

Create a Function to Scrape all the urls

In [32]:
def igdb_parser(url):
    """
    Scrape the content of a single url.
    """
    html = requests.get(url).content
    soup = BeautifulSoup(html,"lxml")
    socials = soup.find_all(['a'],{'class':['gamepage-website-link col-md-4']})
    names = [social.text.strip().split("</a>")[0] for social in socials]
    links = []
    for tag in socials:
        link = tag.get('href')
        if link is not None:
            if link.startswith("http"):
                links.append(link)
    return [names,links]

Test the function

In [33]:
URL_GAME = 'https://www.igdb.com/games/the-sinking-city' 
igdb_parser(URL_GAME)

[['YouTube channel',
  'Facebook page',
  'Twitter profile',
  'Official Website',
  'Wikipedia page',
  'Instagram profile',
  'Subreddit'],
 ['https://www.youtube.com/channel/UCciY3FqSxrfKQ2nMDUrlf4w',
  'https://www.facebook.com/TheSinkingCityGame',
  'https://twitter.com/thesinkingcity',
  'https://thesinkingcity.com/',
  'https://wikipedia.org/wiki/The_Sinking_City',
  'https://www.instagram.com/frogwares',
  'https://www.reddit.com/r/TheSinkingCity']]

Scrape all the Urls

In [34]:
urls_list = games.url
urls_list

0           https://www.igdb.com/games/the-sinking-city
1     https://www.igdb.com/games/fire-emblem-three-h...
2             https://www.igdb.com/games/cyberpunk-2077
3        https://www.igdb.com/games/super-mario-maker-2
4           https://www.igdb.com/games/marvels-avengers
5                     https://www.igdb.com/games/pacify
6            https://www.igdb.com/games/aquapark-dot-io
7               https://www.igdb.com/games/sexy-beach-3
8          https://www.igdb.com/games/cooking-simulator
9       https://www.igdb.com/games/artificial-academy-2
10    https://www.igdb.com/games/eternity-the-last-u...
11             https://www.igdb.com/games/borderlands-3
12             https://www.igdb.com/games/madden-nfl-20
13         https://www.igdb.com/games/children-of-morta
14            https://www.igdb.com/games/ancient-cities
15            https://www.igdb.com/games/dude-simulator
16    https://www.igdb.com/games/a-life-in-silk-the-...
17            https://www.igdb.com/games/legrand

In [38]:
social_list = []
len_sl = len(urls_list)
for i in range(len_sl):
    social_list.append([urls_list[i], igdb_parser(urls_list[i])])
    time.sleep(5) 

In [39]:
social_list

[['https://www.igdb.com/games/the-sinking-city',
  [['YouTube channel',
    'Facebook page',
    'Twitter profile',
    'Official Website',
    'Wikipedia page',
    'Instagram profile',
    'Subreddit'],
   ['https://www.youtube.com/channel/UCciY3FqSxrfKQ2nMDUrlf4w',
    'https://www.facebook.com/TheSinkingCityGame',
    'https://twitter.com/thesinkingcity',
    'https://thesinkingcity.com/',
    'https://wikipedia.org/wiki/The_Sinking_City',
    'https://www.instagram.com/frogwares',
    'https://www.reddit.com/r/TheSinkingCity']]],
 ['https://www.igdb.com/games/fire-emblem-three-houses',
  [['Wikipedia page',
    'Official Website',
    'Twitter profile',
    'YouTube channel'],
   ['https://en.wikipedia.org/wiki/Fire_Emblem:_Three_Houses',
    'https://www.nintendo.com/games/detail/fire-emblem-three-houses-switch',
    'https://twitter.com/fire_houses',
    'https://www.youtube.com/channel/UCGIY_O-8vW4rfX98KlMkvRg']]],
 ['https://www.igdb.com/games/cyberpunk-2077',
  [['Official We

Seleccionar todos los tipos diferentes de redes sociles únicos

In [40]:
social_case = []
for i in range(len_sl):
    for j in range(len(social_list[i][1][0])):
        social_case.append(social_list[i][1][0][j])
        
#Coger sólo los únicos
social_case = list(set(social_case))
social_case

['Wikipedia page',
 'Official Website',
 'Itch',
 'Instagram profile',
 'Subreddit',
 'Facebook page',
 'Twitch channel',
 'Wikia page',
 'YouTube channel',
 'Twitter profile']

In [41]:
#poner las redes sociales como nombre de columna
#y que la primera columna sea url
cols_names = [social for social in social_case]
cols_names.insert(0, "url")
cols_names

['url',
 'Wikipedia page',
 'Official Website',
 'Itch',
 'Instagram profile',
 'Subreddit',
 'Facebook page',
 'Twitch channel',
 'Wikia page',
 'YouTube channel',
 'Twitter profile']

In [42]:
#Create a data frame with the social info of each game
social_df = pd.DataFrame(columns=cols_names )
for i in range(len_sl):
    temp_dic = {}
    temp_dic = dict(zip(social_list[i][1][0],social_list[i][1][1]))
    temp_dic['url'] = social_list[i][0]
    for col in cols_names:
        if col in temp_dic.keys():
            social_df.loc[i,col] = temp_dic[col]
        else:
            social_df.loc[i,col] = np.nan
social_df.head()

,url,Wikipedia page,Official Website,Itch,Instagram profile,Subreddit,Facebook page,Twitch channel,Wikia page,YouTube channel,Twitter profile
0,https://www.igdb.com/games/the-sinking-city,https://wikipedia.org/wiki/The_Sinking_City,https://thesinkingcity.com/,NaN,https://www.instagram.com/frogwares,https://www.reddit.com/r/TheSinkingCity,https://www.facebook.com/TheSinkingCityGame,NaN,NaN,https://www.youtube.com/channel/UCciY3FqSxrfKQ...,https://twitter.com/thesinkingcity
1,https://www.igdb.com/games/fire-emblem-three-h...,https://en.wikipedia.org/wiki/Fire_Emblem:_Thr...,https://www.nintendo.com/games/detail/fire-emb...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCGIY_O-8vW4rf...,https://twitter.com/fire_houses
2,https://www.igdb.com/games/cyberpunk-2077,https://en.wikipedia.org/wiki/Cyberpunk_2077,http://cyberpunk.net/,NaN,NaN,NaN,https://www.facebook.com/CyberpunkGame,NaN,NaN,https://www.youtube.com/user/CyberPunkGame,https://twitter.com/CyberpunkGame
3,https://www.igdb.com/games/super-mario-maker-2,https://en.wikipedia.org/wiki/Super_Mario_Maker_2,https://supermariomaker.nintendo.com/,NaN,NaN,NaN,NaN,NaN,https://www.mariowiki.com/Super_Mario_Maker_2,NaN,NaN
4,https://www.igdb.com/games/marvels-avengers,https://en.wikipedia.org/wiki/Avengers_(2020_v...,https://avengers.square-enix-games.com/,NaN,https://www.instagram.com/PlayAvengers,NaN,https://www.youtube.com/PlayAvengers,NaN,NaN,https://www.youtube.com/channel/UC0CelqHTkd3P7...,https://twitter.com/PlayAvengers


## 3.- UNION OF BOTH DF
Merge both data frames to add the social info to the popular games

We have the most popular games in the games df

In [43]:
games.head()

,id,name,popularity,total_rating,date_time,url
0,18225,The Sinking City,1342.489026,70.119493,2019-06-27,https://www.igdb.com/games/the-sinking-city
1,26845,Fire Emblem: Three Houses,921.876817,93.333333,2019-07-26,https://www.igdb.com/games/fire-emblem-three-h...
2,1877,Cyberpunk 2077,752.805081,NaN,2020-04-16,https://www.igdb.com/games/cyberpunk-2077
3,115276,Super Mario Maker 2,661.996011,88.422354,2019-06-28,https://www.igdb.com/games/super-mario-maker-2
4,26950,Marvel's Avengers,651.855153,NaN,2020-05-15,https://www.igdb.com/games/marvels-avengers


We have the social info of each game in the games df

In [44]:
social_df.head()

,url,Wikipedia page,Official Website,Itch,Instagram profile,Subreddit,Facebook page,Twitch channel,Wikia page,YouTube channel,Twitter profile
0,https://www.igdb.com/games/the-sinking-city,https://wikipedia.org/wiki/The_Sinking_City,https://thesinkingcity.com/,NaN,https://www.instagram.com/frogwares,https://www.reddit.com/r/TheSinkingCity,https://www.facebook.com/TheSinkingCityGame,NaN,NaN,https://www.youtube.com/channel/UCciY3FqSxrfKQ...,https://twitter.com/thesinkingcity
1,https://www.igdb.com/games/fire-emblem-three-h...,https://en.wikipedia.org/wiki/Fire_Emblem:_Thr...,https://www.nintendo.com/games/detail/fire-emb...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCGIY_O-8vW4rf...,https://twitter.com/fire_houses
2,https://www.igdb.com/games/cyberpunk-2077,https://en.wikipedia.org/wiki/Cyberpunk_2077,http://cyberpunk.net/,NaN,NaN,NaN,https://www.facebook.com/CyberpunkGame,NaN,NaN,https://www.youtube.com/user/CyberPunkGame,https://twitter.com/CyberpunkGame
3,https://www.igdb.com/games/super-mario-maker-2,https://en.wikipedia.org/wiki/Super_Mario_Maker_2,https://supermariomaker.nintendo.com/,NaN,NaN,NaN,NaN,NaN,https://www.mariowiki.com/Super_Mario_Maker_2,NaN,NaN
4,https://www.igdb.com/games/marvels-avengers,https://en.wikipedia.org/wiki/Avengers_(2020_v...,https://avengers.square-enix-games.com/,NaN,https://www.instagram.com/PlayAvengers,NaN,https://www.youtube.com/PlayAvengers,NaN,NaN,https://www.youtube.com/channel/UC0CelqHTkd3P7...,https://twitter.com/PlayAvengers


In [45]:
final_games = games.merge(social_df, left_on='url', right_on='url')
final_games.head()

,id,name,popularity,total_rating,date_time,url,Wikipedia page,Official Website,Itch,Instagram profile,Subreddit,Facebook page,Twitch channel,Wikia page,YouTube channel,Twitter profile
0,18225,The Sinking City,1342.489026,70.119493,2019-06-27,https://www.igdb.com/games/the-sinking-city,https://wikipedia.org/wiki/The_Sinking_City,https://thesinkingcity.com/,NaN,https://www.instagram.com/frogwares,https://www.reddit.com/r/TheSinkingCity,https://www.facebook.com/TheSinkingCityGame,NaN,NaN,https://www.youtube.com/channel/UCciY3FqSxrfKQ...,https://twitter.com/thesinkingcity
1,26845,Fire Emblem: Three Houses,921.876817,93.333333,2019-07-26,https://www.igdb.com/games/fire-emblem-three-h...,https://en.wikipedia.org/wiki/Fire_Emblem:_Thr...,https://www.nintendo.com/games/detail/fire-emb...,NaN,NaN,NaN,NaN,NaN,NaN,https://www.youtube.com/channel/UCGIY_O-8vW4rf...,https://twitter.com/fire_houses
2,1877,Cyberpunk 2077,752.805081,NaN,2020-04-16,https://www.igdb.com/games/cyberpunk-2077,https://en.wikipedia.org/wiki/Cyberpunk_2077,http://cyberpunk.net/,NaN,NaN,NaN,https://www.facebook.com/CyberpunkGame,NaN,NaN,https://www.youtube.com/user/CyberPunkGame,https://twitter.com/CyberpunkGame
3,115276,Super Mario Maker 2,661.996011,88.422354,2019-06-28,https://www.igdb.com/games/super-mario-maker-2,https://en.wikipedia.org/wiki/Super_Mario_Maker_2,https://supermariomaker.nintendo.com/,NaN,NaN,NaN,NaN,NaN,https://www.mariowiki.com/Super_Mario_Maker_2,NaN,NaN
4,26950,Marvel's Avengers,651.855153,NaN,2020-05-15,https://www.igdb.com/games/marvels-avengers,https://en.wikipedia.org/wiki/Avengers_(2020_v...,https://avengers.square-enix-games.com/,NaN,https://www.instagram.com/PlayAvengers,NaN,https://www.youtube.com/PlayAvengers,NaN,NaN,https://www.youtube.com/channel/UC0CelqHTkd3P7...,https://twitter.com/PlayAvengers


Export final dataframe in csv

In [71]:
final_games.to_csv("output/final_games.csv", sep=",", header=False, index=False)